# Imports libraries

In [1]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 
# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer #process null values
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import lightgbm as lgb
# system
import itertools
# matplotlib and seaborn for plotting
from sklearn.model_selection import StratifiedKFold, GridSearchCV
import matplotlib.pyplot as plt
import numpy as np
#import seaborn as sns

# Read in data

### Load all datasets

In [2]:
#View the question data
question_data = pd.read_csv('data_all/question.csv')
print(question_data.shape)
question_data.head(5)


(129, 2)


,Column,QuestionText
0,Respondent,Randomized respondent ID number (not in order ...
1,Hobby,Do you code as a hobby?
2,OpenSource,Do you contribute to open source projects?
3,Country,In which country do you currently reside?
4,Student,"Are you currently enrolled in a formal, degree..."


In [3]:
#View the training data
train_data = pd.read_csv('data_all/train.csv')
print(train_data.shape)
train_data.head(5)

(33857, 128)


,No,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,...,Exercise,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS,SurveyTooLong,SurveyEasy
0,1,Yes,No,Nigeria,No,"Independent contractor, freelancer, or self-em...","Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",NaN,Database administrator;Designer;Full-stack dev...,...,I don't typically exercise,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Black or of African descent,35 - 44 years old,Yes,NaN,The survey was too long,Somewhat easy
1,2,Yes,Yes,France,No,Employed full-time,"Master’s degree (MA, MS, M.Eng., MBA, etc.)","A natural science (ex. biology, chemistry, phy...","10,000 or more employees",Back-end developer,...,I don't typically exercise,Male,Straight or heterosexual,"Master’s degree (MA, MS, M.Eng., MBA, etc.)",White or of European descent,35 - 44 years old,Yes,NaN,The survey was too long,Somewhat easy
2,3,Yes,Yes,United Kingdom,No,Employed full-time,Some college/university study without earning ...,"Computer science, computer engineering, or sof...",NaN,Designer;Full-stack developer,...,I don't typically exercise,Male,Straight or heterosexual,"Secondary school (e.g. American high school, G...",White or of European descent,35 - 44 years old,No,NaN,The survey was too long,Somewhat easy
3,4,Yes,No,Russian Federation,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",100 to 499 employees,Back-end developer;Desktop or enterprise appli...,...,I don't typically exercise,Male,Straight or heterosexual,Primary/elementary school,White or of European descent,25 - 34 years old,Yes,NaN,The survey was an appropriate length,Very easy
4,5,No,Yes,Germany,No,"Independent contractor, freelancer, or self-em...",Primary/elementary school,NaN,NaN,Back-end developer;Game or graphics developer,...,I don't typically exercise,Male,Straight or heterosexual,"Secondary school (e.g. American high school, G...",White or of European descent,Under 18 years old,No,NaN,The survey was too short,Very easy


In [4]:
# #Yearly data
# train_yearly_root = train[train['SalaryType'] == 'Yearly'] 
# #generate monthly from yearly
# train_yearly_to_monthly =train_yearly_root
# train_yearly_to_monthly['SalaryType']='Monthly'
# train_yearly_to_monthly['Salary']= train_yearly_root['Salary']/12
# #generate weekly from yearly
# train_yearly_to_weekly =train_yearly_root
# train_yearly_to_weekly['SalaryType']='Weekly'
# train_yearly_to_weekly['Salary']= train_yearly_root['Salary']/54

# #Monthly data
# train_monthly_root = train[train['SalaryType'] == 'Monthly'] 
# #generate yearly from monthly
# train_monthly_to_yearly =train_monthly_root
# train_monthly_to_yearly['SalaryType']='Yearly'
# train_monthly_to_yearly['Salary']= train_monthly_root['Salary']*12
# #generate weekly from monthly
# train_monthly_to_weekly =train_monthly_root
# train_monthly_to_weekly['SalaryType']='Weekly'
# train_monthly_to_weekly['Salary']= train_monthly_root['Salary']/4

# #Weekly data
# train_weekly_root = train[train['SalaryType'] == 'Weekly'] 
# #generate yearly from Weekly
# train_weekly_to_yearly =train_weekly_root
# train_weekly_to_yearly['SalaryType']='Yearly'
# train_weekly_to_yearly['Salary']= train_weekly_root['Salary']*54
# #generate monthly from Weekly
# train_weekly_to_monthly =train_weekly_root
# train_weekly_to_monthly['SalaryType']='Monthly'
# train_weekly_to_monthly['Salary']= train_weekly_root['Salary']*4

In [5]:
# train_data= train.append([train_yearly_to_monthly, train_yearly_to_weekly, train_monthly_to_yearly
#                           ,train_monthly_to_weekly, train_weekly_to_yearly,train_weekly_to_monthly])
# train_data.head(5)

In [6]:
train_data['Salary']

0        10800.0
1        73433.0
2        66672.0
3        31848.0
4            0.0
          ...   
33852    11748.0
33853    20351.0
33854     9393.0
33855    45838.0
33856    93000.0
Name: Salary, Length: 33857, dtype: float64

In [7]:
#View the test data
test_data = pd.read_csv('data_all/test.csv')
#test_data['SalaryType'] = test_data['SalaryType'].fillna('Yearly')
print(test_data.shape)
test_data.head(5)

(11259, 127)


,No,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,...,Exercise,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS,SurveyTooLong,SurveyEasy
0,33858,No,No,United States,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",NaN,Back-end developer;Front-end developer;Full-st...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,33859,Yes,No,United States,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",NaN,100 to 499 employees,Back-end developer;Database administrator;DevO...,...,Daily or almost every day,Male,Straight or heterosexual,"Master’s degree (MA, MS, M.Eng., MBA, etc.)",White or of European descent,25 - 34 years old,Yes,No,The survey was an appropriate length,Very easy
2,33860,Yes,No,United States,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","A natural science (ex. biology, chemistry, phy...",20 to 99 employees,Front-end developer;Full-stack developer,...,I don't typically exercise,"Transgender;Non-binary, genderqueer, or gender...",Bisexual or Queer,"Other doctoral degree (Ph.D, Ed.D., etc.)",White or of European descent,18 - 24 years old,No,No,The survey was an appropriate length,Very easy
3,33861,Yes,Yes,India,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Another engineering discipline (ex. civil, ele...",NaN,Back-end developer,...,Daily or almost every day,Male,Straight or heterosexual,"Other doctoral degree (Ph.D, Ed.D., etc.)",South Asian,18 - 24 years old,No,NaN,The survey was an appropriate length,Somewhat easy
4,33862,Yes,No,Norway,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",Fewer than 10 employees,Front-end developer;Full-stack developer;Mobil...,...,1 - 2 times per week,Male,Straight or heterosexual,"Master’s degree (MA, MS, M.Eng., MBA, etc.)",White or of European descent,35 - 44 years old,Yes,NaN,The survey was too long,Somewhat easy


### Examine the distribution of target data

In [8]:
target = train_data['Salary'].astype(int)

### Examine the missing values on training data

In [9]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [10]:
missing_values = missing_values_table(train_data)
missing_values.head(10)

Your selected dataframe has 128 columns.
There are 122 columns that have missing values.


,Missing Values,% of Total Values
TimeAfterBootcamp,30638,90.5
MilitaryUS,25092,74.1
HackathonReasons,20939,61.8
ErgonomicDevices,18628,55.0
AdBlockerReasons,16431,48.5
StackOverflowJobsRecommend,15239,45.0
JobEmailPriorities5,10841,32.0
JobEmailPriorities7,10841,32.0
JobEmailPriorities6,10841,32.0
JobEmailPriorities4,10841,32.0


In [11]:
missing_row_values = train_data.isnull().sum(axis=1)
missing_row_percentage = 100*missing_row_values/(train_data.shape[1]-1)

In [12]:
missing_row_df = pd.concat([missing_row_values,missing_row_percentage],axis=1)

In [13]:
missing_row_df_ren = missing_row_df.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})

In [14]:
missing_row_df_ren[missing_row_df_ren.iloc[:,1]!=0].sort_values(
        '% of Total Values', ascending=False).round(1).head(10)

,Missing Values,% of Total Values
23584,107,84.3
15718,107,84.3
22301,106,83.5
2173,106,83.5
23209,106,83.5
11628,105,82.7
32908,105,82.7
15283,105,82.7
9642,104,81.9
9843,104,81.9


## Column types

Convert some category columns to numeric

In [15]:
train_data["YearsCoding_convert"] = train_data["YearsCoding"].map( {'0-2 years': 1, '3-5 years': 4,'6-8 years': 7, 
                                                            '9-11 years': 10, '12-14 years': 13, '15-17 years': 16,
                                                           '18-20 years': 19, '21-23 years': 22, '24-26 years': 25,
                                                           '27-29 years': 28, '30 or more years': 30})
test_data["YearsCoding_convert"] = test_data["YearsCoding"].map( {'0-2 years': 1, '3-5 years': 4,'6-8 years': 7, 
                                                            '9-11 years': 10, '12-14 years': 13, '15-17 years': 16,
                                                           '18-20 years': 19, '21-23 years': 22, '24-26 years': 25,
                                                           '27-29 years': 28, '30 or more years': 30})

In [16]:
train_data["YearsCodingProf_convert"] = train_data["YearsCodingProf"].map( {'0-2 years': 1, '3-5 years': 4,'6-8 years': 7, 
                                                            '9-11 years': 10, '12-14 years': 13, '15-17 years': 16,
                                                           '18-20 years': 19, '21-23 years': 22, '24-26 years': 25,
                                                           '27-29 years': 28, '30 or more years': 30})
test_data["YearsCodingProf_convert"] = test_data["YearsCodingProf"].map( {'0-2 years': 1, '3-5 years': 4,'6-8 years': 7, 
                                                            '9-11 years': 10, '12-14 years': 13, '15-17 years': 16,
                                                           '18-20 years': 19, '21-23 years': 22, '24-26 years': 25,
                                                           '27-29 years': 28, '30 or more years': 30})

In [17]:
train_data["StackOverflowRecommend_convert"] = train_data["StackOverflowRecommend"].map( {'10 (Very Likely)': 10,
                                                                                  '0 (Not Likely)': 0})
test_data["StackOverflowRecommend_convert"] = test_data["StackOverflowRecommend"].map( {'10 (Very Likely)': 10,
                                                                                  '0 (Not Likely)': 0})

In [18]:
train_data["StackOverflowJobsRecommend_convert"] = train_data["StackOverflowJobsRecommend"].map( {'10 (Very Likely)': 10,
                                                                                  '0 (Not Likely)': 0})
test_data["StackOverflowJobsRecommend_convert"] = test_data["StackOverflowJobsRecommend"].map( {'10 (Very Likely)': 10,
                                                                                  '0 (Not Likely)': 0})

In [19]:
train_data["HoursComputer_convert"] = train_data["HoursComputer"].map( {'1 - 4 hours': 3,'5 - 8 hours': 7,
                                                               '9 - 12 hours': 11,'Over 12 hours': 14})
test_data["HoursComputer_convert"] = test_data["HoursComputer"].map( {'1 - 4 hours': 3,'5 - 8 hours': 7,
                                                               '9 - 12 hours': 11,'Over 12 hours': 14})

In [20]:
train_data["HoursOutside_convert"] = train_data["HoursOutside"].map( {'Less than 30 minutes': 0.3,'30 - 59 minutes': 0.8,
                                                               '1 - 2 hours': 1.5,'3 - 4 hours': 3.5, 'Over 4 hours': 5})
test_data["HoursOutside_convert"] = test_data["HoursOutside"].map( {'Less than 30 minutes': 0.3,'30 - 59 minutes': 0.8,
                                                               '1 - 2 hours': 1.5,'3 - 4 hours': 3.5, 'Over 4 hours': 5})

In [21]:
train_data["Age_convert"] = train_data["Age"].map( {'18 - 24 years old': 21,'25 - 34 years old': 30,
                                                               '35 - 44 years old': 40,'45 - 54 years old': 50})
test_data["Age_convert"] = test_data["Age"].map( {'18 - 24 years old': 21,'25 - 34 years old': 30,
                                                               '35 - 44 years old': 40,'45 - 54 years old': 50})

In [22]:
train_data["TimeFullyProductive_convert"] = train_data["TimeFullyProductive"].map( {'Less than a month': 1,'One to three months': 3,
                                                               'Six to nine months': 9,'Nine months to a year': 12,
                                                                           'More than a year': 15, 'Three to six months':6})
test_data["TimeFullyProductive_convert"] = test_data["TimeFullyProductive"].map( {'Less than a month': 1,'One to three months': 3,
                                                               'Six to nine months': 9,'Nine months to a year': 12,
                                                                           'More than a year': 15, 'Three to six months':6})

In [23]:
# Number of each type of column
train_data.dtypes.value_counts()

object     86
float64    49
int64       1
dtype: int64

### Examine OBJECT type columns

In [24]:
# Number of unique classes in each object column
train_data.select_dtypes('object').apply(pd.Series.nunique, axis = 0).sort_values(ascending=False).head(10)

LanguageDesireNextYear     14619
LanguageWorkedWith         13583
PlatformDesireNextYear      7501
DatabaseDesireNextYear      5658
PlatformWorkedWith          5249
DevType                     4884
IDE                         4623
DatabaseWorkedWith          4171
FrameworkDesireNextYear     1231
CommunicationTools           935
dtype: int64

In [25]:
#examine the few columns have most of classes
train_data[['LanguageDesireNextYear','LanguageWorkedWith','PlatformDesireNextYear']].head(10)

,LanguageDesireNextYear,LanguageWorkedWith,PlatformDesireNextYear
0,C#;Java;PHP;Python;SQL;HTML;CSS,PHP;SQL;HTML;CSS,Android;Linux;Windows Desktop or Server;WordPress
1,Go;Java;Kotlin,Go;Groovy;Java;JavaScript;Bash/Shell,Linux
2,Haskell,C++;JavaScript;SQL;HTML;CSS,Arduino;Linux;Raspberry Pi
3,Assembly;C;C++,Assembly;C;C++;Go;Python;Bash/Shell,Linux
4,C++;C#;Java;JavaScript;PHP;Python;SQL;HTML;CSS,C#;Java;JavaScript;Lua;PHP;SQL;VB.NET;HTML,Arduino;Gaming console;Linux;Mac OS;Windows De...
5,TypeScript,C#;JavaScript;SQL;HTML;CSS,Android;iOS
6,Rust;Kotlin;HTML;CSS,Java;JavaScript;Rust;Kotlin;HTML;CSS,Amazon Echo;Android;Arduino;AWS;iOS;Raspberry ...
7,Go;Java;JavaScript;PHP;SQL;HTML;CSS,Java;JavaScript;PHP;SQL;HTML;CSS;Bash/Shell,AWS;Serverless
8,JavaScript;PHP;SQL;Bash/Shell,PHP;SQL;Bash/Shell,Amazon Echo;AWS;Serverless
9,C#;JavaScript;PHP;SQL;TypeScript;HTML;CSS,JavaScript;PHP;SQL;HTML;CSS,NaN


### Examine FLOAT type columns

In [26]:
train_data.select_dtypes('float').head(10)

,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,...,AdsPriorities6,AdsPriorities7,YearsCoding_convert,YearsCodingProf_convert,StackOverflowRecommend_convert,StackOverflowJobsRecommend_convert,HoursComputer_convert,HoursOutside_convert,Age_convert,TimeFullyProductive_convert
0,9.0,7.0,8.0,3.0,6.0,10.0,2.0,1.0,5.0,4.0,...,3.0,7.0,19.0,7.0,10.0,NaN,7.0,5.0,40.0,NaN
1,9.0,2.0,1.0,6.0,4.0,8.0,3.0,5.0,10.0,7.0,...,6.0,5.0,19.0,19.0,NaN,NaN,14.0,0.8,40.0,15.0
2,9.0,4.0,6.0,1.0,3.0,5.0,2.0,8.0,10.0,7.0,...,6.0,7.0,28.0,16.0,NaN,NaN,7.0,1.5,40.0,3.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,4.0,10.0,10.0,10.0,10.0,7.0,0.8,30.0,3.0
4,3.0,8.0,10.0,2.0,7.0,9.0,1.0,5.0,6.0,4.0,...,7.0,3.0,7.0,1.0,NaN,NaN,14.0,0.3,NaN,NaN
5,9.0,1.0,8.0,5.0,4.0,6.0,10.0,3.0,7.0,2.0,...,6.0,7.0,7.0,7.0,10.0,10.0,11.0,0.8,30.0,3.0
6,8.0,10.0,9.0,4.0,6.0,2.0,7.0,1.0,3.0,5.0,...,7.0,6.0,7.0,7.0,10.0,10.0,11.0,0.3,30.0,1.0
7,10.0,6.0,4.0,7.0,5.0,1.0,9.0,2.0,8.0,3.0,...,6.0,2.0,19.0,10.0,10.0,NaN,11.0,1.5,30.0,3.0
8,10.0,1.0,9.0,5.0,2.0,3.0,6.0,4.0,7.0,8.0,...,2.0,5.0,13.0,4.0,10.0,NaN,11.0,0.3,30.0,3.0
9,9.0,8.0,7.0,6.0,2.0,1.0,5.0,4.0,10.0,3.0,...,6.0,7.0,10.0,4.0,10.0,NaN,11.0,0.3,21.0,6.0


In [27]:
view=train_data.select_dtypes('float').drop('Salary',axis=1).describe()
view

,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,...,AdsPriorities6,AdsPriorities7,YearsCoding_convert,YearsCodingProf_convert,StackOverflowRecommend_convert,StackOverflowJobsRecommend_convert,HoursComputer_convert,HoursOutside_convert,Age_convert,TimeFullyProductive_convert
count,32390.000000,32390.000000,32390.000000,32390.000000,32390.000000,32390.000000,32390.000000,32390.000000,32390.000000,32390.000000,...,28310.000000,28310.000000,33843.000000,33341.000000,23635.000000,5270.000000,32601.000000,32610.000000,31133.000000,28310.000000
mean,6.601019,6.753226,5.919759,4.027138,3.714078,4.302593,5.610157,4.232664,7.716764,6.122600,...,5.220805,4.803815,10.727152,7.243064,9.961921,8.886148,10.112297,1.259212,31.070568,3.537337
std,2.715798,2.498056,2.643799,2.508112,2.411668,2.453037,2.902357,2.476109,2.377179,2.625833,...,1.826046,1.869364,7.371223,6.576927,0.615920,3.146384,2.447597,0.925838,7.441640,2.712045
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,3.000000,0.300000,21.000000,1.000000
25%,5.000000,5.000000,4.000000,2.000000,2.000000,2.000000,3.000000,2.000000,6.000000,4.000000,...,4.000000,3.000000,4.000000,4.000000,10.000000,10.000000,7.000000,0.800000,30.000000,1.000000
50%,7.000000,7.000000,6.000000,4.000000,3.000000,4.000000,6.000000,4.000000,8.000000,6.000000,...,6.000000,5.000000,10.000000,4.000000,10.000000,10.000000,11.000000,0.800000,30.000000,3.000000
75%,9.000000,9.000000,8.000000,6.000000,5.000000,6.000000,8.000000,6.000000,10.000000,8.000000,...,7.000000,6.000000,16.000000,10.000000,10.000000,10.000000,11.000000,1.500000,30.000000,6.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,7.000000,7.000000,30.000000,30.000000,10.000000,10.000000,14.000000,5.000000,50.000000,15.000000


### Examine INT type columns

In [28]:
train_data.select_dtypes('int').head(10)

""
0
1
2
3
4
5
6
7
8
9


In [29]:
# drop int column type
train_data = train_data.drop('No',axis=1)
test_data = test_data.drop('No',axis=1)

# Data preprocessing

## Clean data

Drop the columns and the rows have more 50% missing values

In [30]:
missing_cols_drop = missing_values[missing_values.iloc[:,1]>80].index

In [31]:
train_data = train_data.drop(missing_cols_drop,axis =1)
test_data = test_data.drop(missing_cols_drop,axis =1)

In [32]:
missing_rows_drop = train_data.isnull().sum(axis=1)

In [33]:
train_data = train_data[missing_rows_drop<(train_data.shape[1]/2)]

In [34]:
print('Training data shape:',train_data.shape)
print('Testing data shape:',test_data.shape)

Training data shape: (33027, 134)
Testing data shape: (11259, 133)


In [35]:
target = train_data['Salary']
train_data = train_data.drop('Salary',axis=1)

## Split dataset to 3 small datasets based on column type

### Float columns type

In [36]:
# get columns name
float_cols_name_train = train_data.select_dtypes('float').columns
float_cols_name_test = test_data.select_dtypes('float').columns

In [37]:
#fill na by median
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
train_arr = imputer.fit_transform(train_data.select_dtypes('float'))
test_arr = imputer.transform(test_data.select_dtypes('float'))

In [38]:
float_dataset_train = pd.DataFrame(train_arr,columns=float_cols_name_train)
float_dataset_test = pd.DataFrame(test_arr,columns=float_cols_name_test)

In [39]:
print('Training float dataset shape: ', float_dataset_train.shape)
print('Testing dloat dataset shape: ', float_dataset_test.shape)

Training float dataset shape:  (33027, 48)
Testing dloat dataset shape:  (11259, 48)


In [40]:
float_dataset_train.head()

,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,...,AdsPriorities6,AdsPriorities7,YearsCoding_convert,YearsCodingProf_convert,StackOverflowRecommend_convert,StackOverflowJobsRecommend_convert,HoursComputer_convert,HoursOutside_convert,Age_convert,TimeFullyProductive_convert
0,9.0,7.0,8.0,3.0,6.0,10.0,2.0,1.0,5.0,4.0,...,3.0,7.0,19.0,7.0,10.0,10.0,7.0,5.0,40.0,3.0
1,9.0,2.0,1.0,6.0,4.0,8.0,3.0,5.0,10.0,7.0,...,6.0,5.0,19.0,19.0,10.0,10.0,14.0,0.8,40.0,15.0
2,9.0,4.0,6.0,1.0,3.0,5.0,2.0,8.0,10.0,7.0,...,6.0,7.0,28.0,16.0,10.0,10.0,7.0,1.5,40.0,3.0
3,7.0,7.0,6.0,4.0,3.0,4.0,6.0,4.0,8.0,6.0,...,7.0,4.0,10.0,10.0,10.0,10.0,7.0,0.8,30.0,3.0
4,3.0,8.0,10.0,2.0,7.0,9.0,1.0,5.0,6.0,4.0,...,7.0,3.0,7.0,1.0,10.0,10.0,14.0,0.3,30.0,3.0


In [41]:
#Check corr
#Checkcorr= float_dataset_train
#Checkcorr['label']=target

In [42]:
# The undersample dataframe
#plt.figure(figsize = (20, 20))
#corr = float_dataset_train.corr()
#sns.heatmap(corr, cmap="coolwarm_r", annot=True)

In [43]:
#corr['label']

### Categorical "multiple-choices" columns type

In [44]:
#function for check which column contains multiple-choice
def check_multiple_choices(text):
    if type(text)==str:
        if ';' in text:
            return 1
        else:
            return 0
    return 0

In [45]:
#Categorical columns is columns, which are not in float cols
categorical_cols_name=[col for col in train_data.columns if col not in float_cols_name_train]
#Multiple choices is categorical with character: ";"
mul_choices_cols_name=[]
for i in categorical_cols_name:
    if sum(train_data[i].apply(check_multiple_choices))>0:
        mul_choices_cols_name.append(i)

In [46]:
#list of columns have multiple-choice on train_dataset
mul_choices_cols_name

['DevType',
 'CommunicationTools',
 'EducationTypes',
 'SelfTaughtTypes',
 'HackathonReasons',
 'LanguageWorkedWith',
 'LanguageDesireNextYear',
 'DatabaseWorkedWith',
 'DatabaseDesireNextYear',
 'PlatformWorkedWith',
 'PlatformDesireNextYear',
 'FrameworkWorkedWith',
 'FrameworkDesireNextYear',
 'IDE',
 'Methodology',
 'VersionControl',
 'AdBlockerReasons',
 'AdsActions',
 'ErgonomicDevices',
 'Gender',
 'SexualOrientation',
 'RaceEthnicity']

In [47]:
#function for count the number of choices on each column
def count_multiple_choice(text):
    if type(text)==str:
        return text.split(";")
    else:
        return []

In [48]:
num_multiple_choice={}
for i in mul_choices_cols_name:
    flat_ls = list(train_data[i].apply(count_multiple_choice))
    num_multiple_choice[i]=len(set(list(itertools.chain(*flat_ls))))

In [49]:
#display the number of choice on columns
#frame_num_choice = pd.DataFrame(num_multiple_choice.items(),columns=['Feature names','Number of choice']).sort_values('Number of choice',ascending=False)
#frame_num_choice

#### Encode multiple-choices to one-hot

In [50]:
#function for encoding multiple-choice to one hot
def encode(ls,ref):
    ini_vec = np.zeros((len(ref)))
    ini_ls=[]
    for i in ls:
        ini_ls.append(ref.index(i))
    ini_vec[ini_ls]=1
    return ini_vec.astype('int')

In [51]:
#apply the function to handle multiple-choices columns
ls_v4_train=[]
ls_v4_test=[]
ls_v2=[]
for c in mul_choices_cols_name:
    
    v1_train = train_data[c].apply(count_multiple_choice) #series
    v1_test = test_data[c].apply(count_multiple_choice) 
    
    v2 = list(set(list(itertools.chain(*v1_train)))) #list of unique of multiple-choices
    v3_train = v1_train.apply(encode,ref=v2) #numpy array
    v3_test = v1_test.apply(encode,ref=v2)
    
    v4_train = np.concatenate(v3_train.values).reshape(-1,len(v2))
    v4_test = np.concatenate(v3_test.values).reshape(-1,len(v2))
    ls_v4_train.append(v4_train)
    ls_v4_test.append(v4_test)
    ls_v2.append(v2)
v5_train = np.concatenate(ls_v4_train,axis=1)
v5_test = np.concatenate(ls_v4_test,axis=1)
v6 = list(itertools.chain(*ls_v2))
multi_choice_dataset_train = pd.DataFrame(v5_train,columns = v6)
multi_choice_dataset_test = pd.DataFrame(v5_test,columns = v6)

In [52]:
test_data['DevType'].head(5)

0    Back-end developer;Front-end developer;Full-st...
1    Back-end developer;Database administrator;DevO...
2             Front-end developer;Full-stack developer
3                                   Back-end developer
4    Front-end developer;Full-stack developer;Mobil...
Name: DevType, dtype: object

In [53]:
multi_choice_dataset_train.iloc[:5,:20]

,Product manager,Engineering manager,Data or business analyst,System administrator,Back-end developer,Game or graphics developer,Data scientist or machine learning specialist,Designer,Database administrator,Front-end developer,DevOps specialist,Full-stack developer,Marketing or sales professional,QA or test developer,Embedded applications or devices developer,Educator or academic researcher,"C-suite executive (CEO, CTO, etc.)",Desktop or enterprise applications developer,Student,Mobile developer
0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0
4,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [54]:
print('Training multiple-choices dataset shape: ', multi_choice_dataset_train.shape)
print('Testing multiple-choices dataset shape: ', multi_choice_dataset_test.shape)

Training multiple-choices dataset shape:  (33027, 320)
Testing multiple-choices dataset shape:  (11259, 320)


### Categorical "option-choices" columns type

In [55]:
option_choices_cols_name=[col for col in categorical_cols_name if col not in mul_choices_cols_name]

In [56]:
option_choices_cols_name

['Hobby',
 'OpenSource',
 'Country',
 'Student',
 'Employment',
 'FormalEducation',
 'UndergradMajor',
 'CompanySize',
 'YearsCoding',
 'YearsCodingProf',
 'JobSatisfaction',
 'CareerSatisfaction',
 'HopeFiveYears',
 'JobSearchStatus',
 'LastNewJob',
 'UpdateCV',
 'Currency',
 'SalaryType',
 'CurrencySymbol',
 'TimeFullyProductive',
 'AgreeDisagree1',
 'AgreeDisagree2',
 'AgreeDisagree3',
 'OperatingSystem',
 'NumberMonitors',
 'CheckInCode',
 'AdBlocker',
 'AdBlockerDisable',
 'AdsAgreeDisagree1',
 'AdsAgreeDisagree2',
 'AdsAgreeDisagree3',
 'AIDangerous',
 'AIInteresting',
 'AIResponsible',
 'AIFuture',
 'EthicsChoice',
 'EthicsReport',
 'EthicsResponsible',
 'EthicalImplications',
 'StackOverflowRecommend',
 'StackOverflowVisit',
 'StackOverflowHasAccount',
 'StackOverflowParticipate',
 'StackOverflowJobs',
 'StackOverflowDevStory',
 'StackOverflowJobsRecommend',
 'StackOverflowConsiderMember',
 'HypotheticalTools1',
 'HypotheticalTools2',
 'HypotheticalTools3',
 'HypotheticalTools4

In [57]:
#impute missing value by majority class
imputer = SimpleImputer(missing_values=np.nan,strategy='most_frequent')
train_data_arr = imputer.fit_transform(train_data[option_choices_cols_name])
test_data_arr = imputer.transform(test_data[option_choices_cols_name])

In [58]:
option_choices_dataset_train = pd.DataFrame(train_data_arr,columns=option_choices_cols_name)
option_choices_dataset_test = pd.DataFrame(test_data_arr,columns=option_choices_cols_name)

#### Encode option-choices to one-hot columns

In [59]:
# Create a label encoder for option columns
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in option_choices_cols_name:
    # If 2 or fewer unique categories
    if len(list(option_choices_dataset_train[col].unique())) <= 2:
        # Train on the training data
        le.fit(option_choices_dataset_train[col])
        # Transform both training and testing data
        option_choices_dataset_train[col] = le.transform(option_choices_dataset_train[col])
        option_choices_dataset_test[col] = le.transform(option_choices_dataset_test[col])    
        # Keep track of how many columns were label encoded
        le_count += 1
            
print('%d columns were label encoded.' % le_count)

4 columns were label encoded.


In [60]:
#one-hot encode for train_data
option_choices_dataset_train = pd.get_dummies(option_choices_dataset_train)
option_choices_dataset_train.shape

(33027, 587)

In [61]:
#one-hot encode for test_data
option_choices_dataset_test = pd.get_dummies(option_choices_dataset_test)
option_choices_dataset_test.shape

(11259, 547)

In [62]:
#Align the training and testing data, keep only columns present in both dataframes
option_choices_dataset_train, option_choices_dataset_test = option_choices_dataset_train.align(option_choices_dataset_test, join = 'inner', axis = 1)
print('Training option-choices dataset shape: ', option_choices_dataset_train.shape)
print('Testing option-choices dataset shape: ', option_choices_dataset_test.shape)

Training option-choices dataset shape:  (33027, 537)
Testing option-choices dataset shape:  (11259, 537)


### Concate all dataframe

In [63]:
#float_dataset_train,
train_dataset = pd.concat([float_dataset_train,multi_choice_dataset_train,option_choices_dataset_train],axis=1)

In [64]:
#float_dataset_test,
test_dataset = pd.concat([float_dataset_test,multi_choice_dataset_test,option_choices_dataset_test],axis=1)

In [65]:
print('Training dataset shape: ', train_dataset.shape)
#print('Training label shape: ', y_train.shape)
print('Testing dataset shape: ', test_dataset.shape)

Training dataset shape:  (33027, 905)
Testing dataset shape:  (11259, 905)


# Modeling

In [66]:
from sklearn.utils.validation import column_or_1d

In [67]:
X_train= train_dataset
y_train= target.astype(int)
X_test= test_dataset

In [68]:
y_train.dtype

dtype('int32')

In [69]:
import re
X_train = X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
X_test = X_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

### Split training set to validation set

In [70]:
params = {
    'objective'         : 'regression',    
    'metric'            : 'l2', 
    'nthread'           : 4,
    'learning_rate'     : 0.001,

    'num_leaves'        : 23,
    'feature_fraction'  : 0.106,
    'bagging_fraction'  : 0.825,
    'max_depth'         : -1,
    'lambda_l1'         : 0.2,
    'lambda_l2'         : 2.7,
    'min_split_gain'    : 0.007,
}

# # Create parameters to search
# gridParams = {
#     'learning_rate': [0.005,0.0005],
#     'num_leaves': [12,16,23,25],
#     'lambda_l1' : [0.1,0.2,0.3],
#     'lambda_l2' : [2.5,2.7,2.9],
#     }

# # Create REGRESSOR to use. Note that parameters have to be input manually
# # not as a dict!
# mdl = lgb.LGBMRegressor(
#           objective = params['objective'],
#           metric = params['metric'],
#           nthread = params['nthread'],
#           learning_rate = params['learning_rate'],
#           num_leaves = params['num_leaves'],
#           feature_fraction = params['feature_fraction'],
#           bagging_fraction = params['bagging_fraction'],
#           max_depth = params['max_depth'],
#           lambda_l1 = params['lambda_l1'],
#           lambda_l2 = params['lambda_l2'],
#           min_split_gain = params['min_split_gain'])

# # To view the default model params:
# mdl.get_params().keys()

# # Create the grid
# grid = GridSearchCV(mdl, gridParams,
#                     verbose=0,
#                     cv=4,
#                     n_jobs=2)

# # Run the grid
# grid.fit(X_train, y_train)

# # Print the best parameters found
# print(grid.best_params_)
# print(grid.best_score_)

# # Using parameters already set above, replace in the best from the grid search
# params['learning_rate'] = grid.best_params_['learning_rate']
# params['num_leaves'] = grid.best_params_['num_leaves']
# params['lambda_l1'] = grid.best_params_['lambda_l1']
# params['lambda_l2'] = grid.best_params_['lambda_l2']
# # params['subsample_for_bin'] = grid.best_params_['subsample_for_bin']

# print('Fitting with params: ')
# print(params)

In [71]:
skf = StratifiedKFold(n_splits=4, random_state=3462873, shuffle=True)
preds = 0.0
for itrain, ivalid in skf.split(X_train, y_train):
    lgb_train = lgb.Dataset(X_train.iloc[itrain], y_train.iloc[itrain])
    lgb_eval  = lgb.Dataset(X_train.iloc[ivalid], y_train.iloc[ivalid], reference = lgb_train)
    model = lgb.train(params,
                lgb_train,
                num_boost_round = 999999,  
                early_stopping_rounds = 800,
                verbose_eval = False,
                valid_sets = [lgb_train, lgb_eval])
    pred = model.predict(X_test)
    preds += pred/skf.n_splits

c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  % (min_groups, self.n_splits)), UserWarning)


# Submit

In [72]:
def to_submit_file(file_name, probs):
    """
    A helper function helps generating submit files
    
    :param file_name: submit file name
    :param probs: list of probabilities for test samples
    
    :return: None
    """
    submit = test_id.copy()
    submit['Salary'] = probs
    submit.to_csv(file_name, index = False)
    print ('Done writting submit file: ', file_name)

In [73]:
test_id = pd.read_csv('data_all/submit.csv')

In [74]:
y_pred = preds

In [75]:
to_submit_file('submitfile.csv',y_pred)

Done writting submit file:  submitfile.csv
